Load the `pandas` and `sqlite3` libraries and establish a connection to `FinalDB.db`

In [1]:
import csv, sqlite3

con = sqlite3.connect("FinalDB.db")
cur = con.cursor()

Load the SQL magic module and establish a connection with the FinalDB.db

In [2]:
%load_ext sql

%sql sqlite:///FinalDB.db

Use `Pandas` to load the data available in the links above to dataframes. Use these dataframes to load data on to the database `FinalDB.db` as required tables.


In [3]:
import pandas as pd
df = pd.read_csv('C:/Users/mica_/OneDrive/Escritorio/Cursos/Anàlisi de dades BA/Prácticas/IMB SQL Data Science/ChicagoPublicSchools.csv')
df.to_sql("CHICAGO_PUBLIC_SCHOOLS", con, if_exists='replace', index=False)

566

In [4]:
df = pd.read_csv('C:/Users/mica_/OneDrive/Escritorio/Cursos/Anàlisi de dades BA/Prácticas/IMB SQL Data Science/ChicagoCensusData.csv')
df.to_sql("CENSUS_DATA", con, if_exists='replace', index=False)

78

In [5]:
df = pd.read_csv('C:/Users/mica_/OneDrive/Escritorio/Cursos/Anàlisi de dades BA/Prácticas/IMB SQL Data Science/ChicagoCrimeData.csv')
df.to_sql("CHICAGO_CRIME_DATA", con, if_exists='replace', index=False)

533

### 1) Find the total number of crimes recorded in the CRIME table.

In [6]:
%%sql
SELECT COUNT(*)
FROM CHICAGO_CRIME_DATA

 * sqlite:///FinalDB.db
Done.


COUNT(*)
533


### 2) List community area names and numbers with per capita income less than 11000

In [11]:
%%sql
SELECT COMMUNITY_AREA_NUMBER, COMMUNITY_AREA_NAME, PER_CAPITA_INCOME
FROM CENSUS_DATA
WHERE per_capita_income < 11000

 * sqlite:///FinalDB.db
Done.


COMMUNITY_AREA_NUMBER,COMMUNITY_AREA_NAME,PER_CAPITA_INCOME
26.0,West Garfield Park,10934
30.0,South Lawndale,10402
37.0,Fuller Park,10432
54.0,Riverdale,8201


### 3) List all case numbers for crimes involving minors?(children are not considered minors for the purposes of crime analysis)

In [13]:
%%sql
SELECT CASE_NUMBER, DESCRIPTION
FROM CHICAGO_CRIME_DATA
WHERE DESCRIPTION LIKE '%CHILD%'

 * sqlite:///FinalDB.db
Done.


CASE_NUMBER,DESCRIPTION
HN567387,AGG SEX ASSLT OF CHILD FAM MBR
HR391350,SEX ASSLT OF CHILD BY FAM MBR
HN144152,CHILD ABDUCTION/STRANGER


### 4) List all kidnapping crimes involving a child?

In [19]:
%%sql
SELECT CASE_NUMBER, DESCRIPTION
FROM CHICAGO_CRIME_DATA
WHERE DESCRIPTION LIKE '%CHILD%' AND DESCRIPTION LIKE '%ABDUCTION%' OR DESCRIPTION LIKE '%KIDNAPPING%'

 * sqlite:///FinalDB.db
Done.


CASE_NUMBER,DESCRIPTION
HN144152,CHILD ABDUCTION/STRANGER


### 5) List the kind of crimes that were recorded at schools. (No repetitions)

In [25]:
%%sql
SELECT DESCRIPTION, LOCATION_DESCRIPTION
FROM CHICAGO_CRIME_DATA
WHERE LOCATION_DESCRIPTION LIKE '%SCHOOL%'
GROUP BY DESCRIPTION

 * sqlite:///FinalDB.db
Done.


DESCRIPTION,LOCATION_DESCRIPTION
BOMB THREAT,"SCHOOL, PRIVATE, BUILDING"
MANU/DEL:CANNABIS 10GM OR LESS,"SCHOOL, PUBLIC, BUILDING"
POSS: HEROIN(WHITE),"SCHOOL, PUBLIC, GROUNDS"
PRO EMP HANDS NO/MIN INJURY,"SCHOOL, PUBLIC, BUILDING"
SIMPLE,"SCHOOL, PUBLIC, GROUNDS"
TO LAND,"SCHOOL, PUBLIC, GROUNDS"
TO VEHICLE,"SCHOOL, PUBLIC, GROUNDS"


### 6) List the type of schools along with the average safety score for each type.

In [29]:
%%sql 
SELECT "Elementary, Middle, or High School", ROUND(AVG(SAFETY_SCORE),2) AS AVG_SAFETY_sCORE
FROM CHICAGO_PUBLIC_sCHOOLS
GROUP BY "Elementary, Middle, or High School"

 * sqlite:///FinalDB.db
Done.


"Elementary, Middle, or High School",AVG_SAFETY_sCORE
ES,49.52
HS,49.62
MS,48.0


### 7) List 5 community areas with highest % of households below poverty line

In [31]:
%%sql
SELECT COMMUNITY_AREA_NAME, PERCENT_HOUSEHOLDS_BELOW_POVERTY
FROM CENSUS_DATA
ORDER BY PERCENT_HOUSEHOLDS_BELOW_POVERTY DESC
LIMIT 5


 * sqlite:///FinalDB.db
Done.


COMMUNITY_AREA_NAME,PERCENT_HOUSEHOLDS_BELOW_POVERTY
Riverdale,56.5
Fuller Park,51.2
Englewood,46.6
North Lawndale,43.1
East Garfield Park,42.4


### 8) Which community area is most crime prone? Display the coumminty area number only.

In [37]:
%%sql
SELECT COMMUNITY_AREA_NUMBER	
FROM CHICAGO_CRIME_DATA
GROUP BY COMMUNITY_AREA_NUMBER
ORDER BY COUNT(*) DESC
LIMIT 1

 * sqlite:///FinalDB.db
Done.


COMMUNITY_AREA_NUMBER
25.0


### 9) Use a sub-query to find the name of the community area with highest hardship index

In [45]:
%%sql
SELECT COMMUNITY_AREA_NAME
FROM CENSUS_DATA
WHERE HARDSHIP_INDEX = (SELECT MAX(HARDSHIP_INDEX)
                        FROM CENSUS_DATA)

 * sqlite:///FinalDB.db
Done.


COMMUNITY_AREA_NAME
Riverdale


### 10) Use a sub-query to determine the Community Area Name with most number of crimes?

In [53]:
%%sql
SELECT COMMUNITY_AREA_NAME
FROM CENSUS_DATA CD
WHERE COMMUNITY_AREA_NUMBER IN (SELECT COMMUNITY_AREA_NUMBER	
                                FROM CHICAGO_CRIME_DATA
                                GROUP BY COMMUNITY_AREA_NUMBER
                                ORDER BY COUNT(*) DESC
                                LIMIT 1)


 * sqlite:///FinalDB.db
Done.


COMMUNITY_AREA_NAME
Austin
